<a href="https://colab.research.google.com/github/mintycake420/Basic-Exercises-for-courses/blob/main/InformationRetreival_EX05_211718366.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval Assignment 5: Query Expansion with Crusade Wikipedia Articles

##This notebook implements:
- Document collection and chunking (30-word units)
- Stop words removal with a 10 word window
- Query expansion using word embeddings library- Neural networks with word2vec
- Retrieval results comparison

Queries chosen:

**1. crusade battle Jerusalem**

**2. Saladin military victory**

**3. siege fortress attack**

**4. knights templar warrior**

**5. holy land conquest**

____________________________________






**Submitted by:** Yotam Katz  
**Date:** December 2025  
**Course:** אחזור מידע 26 3700 א01  
**Lecturer:** Dr. Moshe Friedman  
**ID:** 211718366  
**Email:** Yotamkatz2000@gmail.com

## 1. Setup and Imports

In [ ]:
# Install required packages
!pip install wikipedia-api gensim nltk

import wikipediaapi
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim.downloader as api
from gensim.models import Word2Vec
import re
from collections import defaultdict
import pandas as pd

# Download NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

print("Setup complete!")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 78.7 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=fb99525fc4489276b7454fdd2402f241f7008361f2d7df9baf4e3ed294b59a9f
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Setup complete!


## 2. Part א.1: Data Collection - Crusade Wikipedia Articles

In [ ]:
# List of Crusade-related Wikipedia articles
crusade_topics = [
    "First Crusade",
    "Second Crusade",
    "Third Crusade",
    "Fourth Crusade",
    "Fifth Crusade",
    "Sixth Crusade",
    "Seventh Crusade",
    "Eighth Crusade",
    "Ninth Crusade",
    "Kingdom of Jerusalem",
    "Saladin",
    "Richard I of England",
    "Siege of Jerusalem (1099)",
    "Battle of Hattin",
    "Knights Templar",
    "Knights Hospitaller",
    "Godfrey of Bouillon",
    "Baldwin I of Jerusalem",
    "Siege of Acre (1189–1191)",
    "People's Crusade"
]

# Initialize Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='CrusadeIRProject/1.0 (Educational)',
    language='en'
)

# Collect documents
documents = {}
for topic in crusade_topics:
    page = wiki_wiki.page(topic)
    if page.exists():
        documents[topic] = page.text
        print(f"✓ Collected: {topic} ({len(page.text)} chars)")
    else:
        print(f"✗ Not found: {topic}")

print(f"\nTotal documents collected: {len(documents)}")

✓ Collected: First Crusade (74322 chars)
✓ Collected: Second Crusade (31670 chars)
✓ Collected: Third Crusade (46997 chars)
✓ Collected: Fourth Crusade (65418 chars)
✓ Collected: Fifth Crusade (45665 chars)
✓ Collected: Sixth Crusade (39125 chars)
✓ Collected: Seventh Crusade (70637 chars)
✓ Collected: Eighth Crusade (45880 chars)
✓ Collected: Ninth Crusade (14566 chars)
✓ Collected: Kingdom of Jerusalem (89758 chars)
✓ Collected: Saladin (73392 chars)
✓ Collected: Richard I of England (52737 chars)
✓ Collected: Siege of Jerusalem (1099) (16616 chars)
✓ Collected: Battle of Hattin (17177 chars)
✓ Collected: Knights Templar (42347 chars)
✓ Collected: Knights Hospitaller (52582 chars)
✓ Collected: Godfrey of Bouillon (21201 chars)
✓ Collected: Baldwin I of Jerusalem (41056 chars)
✓ Collected: Siege of Acre (1189–1191) (23425 chars)
✓ Collected: People's Crusade (12219 chars)

Total documents collected: 20


## 3. Document Chunking - Split into 30-word Units

In [ ]:
def chunk_document(text, chunk_size=30):
    """
    Split document into chunks of specified word count.

    Args:
        text: Document text
        chunk_size: Number of words per chunk (default 30)

    Returns:
        List of text chunks
    """
    # Clean text: remove multiple spaces, newlines
    text = re.sub(r'\s+', ' ', text).strip()

    # Split into words
    words = text.split()

    # Create chunks
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)

    return chunks

# Create document chunks with metadata
doc_chunks = []
for doc_title, doc_text in documents.items():
    chunks = chunk_document(doc_text, chunk_size=30)
    for idx, chunk in enumerate(chunks):
        doc_chunks.append({
            'doc_title': doc_title,
            'chunk_id': idx,
            'text': chunk
        })

print(f"Total chunks created: {len(doc_chunks)}")
print(f"\nExample chunk:")
print(f"Document: {doc_chunks[0]['doc_title']}")
print(f"Chunk {doc_chunks[0]['chunk_id']}: {doc_chunks[0]['text'][:200]}...")

Total chunks created: 4822

Example chunk:
Document: First Crusade
Chunk 0: The First Crusade (1096–1099) was the first and most successful of a series of religious wars, or Crusades, which were initiated, supported and at times directed by the Latin Church...


## 4. Part 2 Stop Words Removal

In [ ]:
# Load English stop words
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    """
    Remove stop words from text, keeping only content words.
    Also removes punctuation and converts to lowercase.

    Args:
        text: Input text

    Returns:
        List of content words
    """
    # Tokenize and lowercase
    words = word_tokenize(text.lower())

    # Remove stop words and non-alphabetic tokens
    content_words = [word for word in words
                     if word.isalpha() and word not in stop_words]

    return content_words

# Process all chunks - remove stop words
for chunk in doc_chunks:
    chunk['content_words'] = remove_stopwords(chunk['text'])
    chunk['content_text'] = ' '.join(chunk['content_words'])

print("Stop words removal complete!")
print(f"\nExample:")
print(f"Original: {doc_chunks[10]['text'][:150]}...")
print(f"\nAfter removing stop words: {doc_chunks[10]['content_text'][:150]}...")
print(f"\nOriginal word count: {len(doc_chunks[10]['text'].split())}")
print(f"Content word count: {len(doc_chunks[10]['content_words'])}")

Stop words removal complete!

Example:
Original: and Adhemar of Le Puy; men from Upper and Lower Lorraine led by Godfrey of Bouillon and his brother Baldwin of Boulogne; Italo-Norman forces led by Bo...

After removing stop words: adhemar le puy men upper lower lorraine led godfrey bouillon brother baldwin boulogne forces led bohemond taranto...

Original word count: 30
Content word count: 17


## 5. Load Pre-trained Word Embeddings

In [ ]:
# Load pre-trained Word2Vec model (Google News vectors)
# This may take a few minutes to download
print("Loading pre-trained Word2Vec model (this may take a few minutes)...")
word2vec_model = api.load('word2vec-google-news-300')
print("✓ Model loaded successfully!")

# Test the model
print("\nTest - Most similar words to 'crusade':")
try:
    similar_words = word2vec_model.most_similar('crusade', topn=5)
    for word, similarity in similar_words:
        print(f"  {word}: {similarity:.3f}")
except KeyError:
    print("  Word not in vocabulary")

Loading pre-trained Word2Vec model (this may take a few minutes)...
[==================================================] 100.0% 1662.8/1662.8MB downloaded
✓ Model loaded successfully!

Test - Most similar words to 'crusade':
  crusades: 0.685
  crusader: 0.602
  quest: 0.544
  crusading: 0.536
  crusaders: 0.527


## 6. Part 3: Define Queries (Content Words Only)

In [ ]:
# Define queries
queries = [
    "crusade battle Jerusalem",
    "Saladin military victory",
    "siege fortress attack",
    "knights templar warrior",
    "holy land conquest"
]

print("Queries defined:")
for i, query in enumerate(queries, 1):
    print(f"{i}. {query}")

Queries defined:
1. crusade battle Jerusalem
2. Saladin military victory
3. siege fortress attack
4. knights templar warrior
5. holy land conquest


## 7. Part 4: Query Expansion

In [ ]:
def expand_query(query, model, top_n=3):
    """
    Expand query by finding similar words for each term.

    Args:
        query: Original query string
        model: Word2Vec model
        top_n: Number of similar words to add per term

    Returns:
        Dictionary with original terms and their expansions
    """
    query_terms = query.lower().split()
    expanded = {}
    all_expanded_terms = set(query_terms)  # Start with original terms

    for term in query_terms:
        try:
            # Find similar words
            similar = model.most_similar(term, topn=top_n)
            similar_words = [word for word, _ in similar]
            expanded[term] = similar_words
            all_expanded_terms.update(similar_words)
        except KeyError:
            # Word not in vocabulary
            expanded[term] = []
            print(f"  Warning: '{term}' not in vocabulary")

    return expanded, list(all_expanded_terms)

# Expand all queries
expanded_queries = {}
for query in queries:
    print(f"\nExpanding query: '{query}'")
    expansion_details, all_terms = expand_query(query, word2vec_model, top_n=3)
    expanded_queries[query] = {
        'original_terms': query.split(),
        'expansion_details': expansion_details,
        'all_expanded_terms': all_terms
    }

    # Display expansion
    for original_term, similar_terms in expansion_details.items():
        print(f"  {original_term} → {', '.join(similar_terms)}")
    print(f"  Final expanded query: {' '.join(all_terms)}")


Expanding query: 'crusade battle Jerusalem'
  crusade → crusades, crusader, quest
  battle → battles, tussle, fight
  jerusalem → palestine, palestinian, lebanon
  Final expanded query: lebanon fight tussle battle palestine crusade jerusalem crusader battles palestinian quest crusades

Expanding query: 'Saladin military victory'
  saladin → 
  military → armed_forces, army, miltary
  victory → triumph, win, victories
  Final expanded query: win military armed_forces triumph victory saladin miltary army victories

Expanding query: 'siege fortress attack'
  siege → seige, blockade, sieges
  fortress → fortresses, citadel, medieval_fortress
  attack → attacks, attck, atack
  Final expanded query: attack atack seige sieges fortress attck medieval_fortress blockade attacks siege citadel fortresses

Expanding query: 'knights templar warrior'
  knights → knight, shinning_armor, noblemen
  templar → templars, halfling, dwarven
  warrior → warriors, anti_warrior_Markin, mage_rogue
  Final expa

## 8. Retrieval Function - Simple Term Matching

In [ ]:
def retrieve_documents(query_terms, doc_chunks, top_k=None):
    """
    Simple retrieval based on term matching.

    Args:
        query_terms: List of query terms
        doc_chunks: List of document chunks
        top_k: Number of top results to return (None = return all)

    Returns:
        List of (chunk, score) tuples
    """
    results = []
    query_terms_set = set(term.lower() for term in query_terms)

    for chunk in doc_chunks:
        # Count matching terms
        chunk_words = set(chunk['content_words'])
        matches = query_terms_set.intersection(chunk_words)
        score = len(matches)

        if score > 0:
            results.append({
                'chunk': chunk,
                'score': score,
                'matches': matches
            })

    # Sort by score (descending)
    results.sort(key=lambda x: x['score'], reverse=True)

    # Return top_k or all results
    if top_k is not None:
        return results[:top_k]
    return results

print("Retrieval function defined!")

Retrieval function defined!


## 9. Part ג: Display Results - Compare Original vs Expanded Queries

In [ ]:
def display_results(query, original_results, expanded_results):
    """
    Display and compare retrieval results.
    """
    print("="*100)
    print(f"QUERY: {query}")
    print("="*100)

    print(f"\n📊 STATISTICS:")
    print(f"  Original query results: {len(original_results)}")
    print(f"  Expanded query results: {len(expanded_results)}")
    print(f"  Improvement: +{len(expanded_results) - len(original_results)} documents")
    if len(original_results) > 0:
        improvement_pct = (len(expanded_results) - len(original_results)) / len(original_results) * 100
        print(f"  Percentage improvement: {improvement_pct:.1f}%")

    print(f"\n🔍 TOP 5 RESULTS - ORIGINAL QUERY:")
    print("-" * 100)
    if len(original_results) == 0:
        print("  No results found!")
    for i, result in enumerate(original_results[:5], 1):
        chunk = result['chunk']
        print(f"\n{i}. Score: {result['score']} | Document: {chunk['doc_title']} | Chunk: {chunk['chunk_id']}")
        print(f"   Matches: {', '.join(result['matches'])}")
        print(f"   Text: {chunk['text'][:200]}...")

    print(f"\n\n✨ TOP 5 RESULTS - EXPANDED QUERY:")
    print("-" * 100)
    if len(expanded_results) == 0:
        print("  No results found!")
    for i, result in enumerate(expanded_results[:5], 1):
        chunk = result['chunk']
        print(f"\n{i}. Score: {result['score']} | Document: {chunk['doc_title']} | Chunk: {chunk['chunk_id']}")
        print(f"   Matches: {', '.join(result['matches'])}")
        print(f"   Text: {chunk['text'][:200]}...")

    print("\n" + "="*100 + "\n")

# Run retrieval for all queries
# IMPORTANT: We retrieve ALL results (no top_k limit) to see true improvement
for query in queries:
    # Original query - GET ALL RESULTS
    original_terms = query.split()
    original_results = retrieve_documents(original_terms, doc_chunks, top_k=None)

    # Expanded query - GET ALL RESULTS
    expanded_terms = expanded_queries[query]['all_expanded_terms']
    expanded_results = retrieve_documents(expanded_terms, doc_chunks, top_k=None)

    # Display comparison
    display_results(query, original_results, expanded_results)

QUERY: crusade battle Jerusalem

📊 STATISTICS:
  Original query results: 1204
  Expanded query results: 1505
  Improvement: +301 documents
  Percentage improvement: 25.0%

🔍 TOP 5 RESULTS - ORIGINAL QUERY:
----------------------------------------------------------------------------------------------------

1. Score: 3 | Document: Battle of Hattin | Chunk: 3
   Matches: battle, crusade, jerusalem
   Text: Jerusalem and most of the other Crusader-held cities and castles. These Christian defeats prompted the Third Crusade, which began two years after the Battle of Hattin. Location The battle took...

2. Score: 2 | Document: First Crusade | Chunk: 8
   Matches: battle, crusade
   Text: at the Battle of Civetot in October 1096. In what has become known as the Princes' Crusade, members of the high nobility and their followers embarked in late-summer 1096 and...

3. Score: 2 | Document: First Crusade | Chunk: 16
   Matches: battle, crusade
   Text: of the residents were massacred. A Fatimid c

## 10. Summary Statistics

In [ ]:
# Calculate summary statistics
# IMPORTANT: Get ALL results to see true improvement
summary_data = []

for query in queries:
    original_terms = query.split()
    expanded_terms = expanded_queries[query]['all_expanded_terms']

    # Retrieve ALL matching documents (no limit)
    original_results = retrieve_documents(original_terms, doc_chunks, top_k=None)
    expanded_results = retrieve_documents(expanded_terms, doc_chunks, top_k=None)

    improvement = len(expanded_results) - len(original_results)
    improvement_pct = (improvement / len(original_results) * 100) if len(original_results) > 0 else 0

    summary_data.append({
        'Query': query,
        'Original Terms': len(original_terms),
        'Expanded Terms': len(expanded_terms),
        'Original Results': len(original_results),
        'Expanded Results': len(expanded_results),
        'Improvement': improvement,
        'Improvement %': f"{improvement_pct:.1f}%"
    })

# Create DataFrame
summary_df = pd.DataFrame(summary_data)

print("\n" + "="*100)
print("SUMMARY: Query Expansion Effectiveness (ALL RESULTS - NO LIMIT)")
print("="*100)
print(summary_df.to_string(index=False))
print("\n" + "="*100)

# Overall statistics
total_original = summary_df['Original Results'].sum()
total_expanded = summary_df['Expanded Results'].sum()
total_improvement = summary_df['Improvement'].sum()

print(f"\n📈 OVERALL STATISTICS:")
print(f"  Average improvement: +{summary_df['Improvement'].mean():.1f} documents per query")
print(f"  Total original results: {total_original}")
print(f"  Total expanded results: {total_expanded}")
print(f"  Overall improvement: +{total_improvement} documents")
if total_original > 0:
    overall_pct = (total_improvement / total_original * 100)
    print(f"  Overall percentage increase: {overall_pct:.1f}%")


SUMMARY: Query Expansion Effectiveness (ALL RESULTS - NO LIMIT)
                   Query  Original Terms  Expanded Terms  Original Results  Expanded Results  Improvement Improvement %
crusade battle Jerusalem               3              12              1204              1505          301         25.0%
Saladin military victory               3               9               659               963          304         46.1%
   siege fortress attack               3              12               364               458           94         25.8%
 knights templar warrior               3              12               326               446          120         36.8%
      holy land conquest               3              12               385               458           73         19.0%


📈 OVERALL STATISTICS:
  Average improvement: +178.4 documents per query
  Total original results: 2938
  Total expanded results: 3830
  Overall improvement: +892 documents
  Overall percentage increase: 30.4%


## 11. Detailed Analysis: What Words Were Added?

In [ ]:
# Analyze which expanded terms contributed to new results
print("\n" + "="*100)
print("DETAILED ANALYSIS: Contribution of Expanded Terms")
print("="*100)

for query in queries:
    print(f"\n{'='*100}")
    print(f"Query: {query}")
    print(f"{'='*100}")

    original_terms = set(query.lower().split())
    expanded_terms = set(expanded_queries[query]['all_expanded_terms'])
    new_terms = expanded_terms - original_terms

    print(f"\nOriginal terms: {', '.join(original_terms)}")
    print(f"New terms added: {', '.join(new_terms)}")

    # Count matches for each new term
    term_contribution = {}
    for term in new_terms:
        count = 0
        for chunk in doc_chunks:
            if term in chunk['content_words']:
                count += 1
        term_contribution[term] = count

    # Sort by contribution
    sorted_terms = sorted(term_contribution.items(), key=lambda x: x[1], reverse=True)

    print(f"\nContribution of each expanded term:")
    for term, count in sorted_terms:
        print(f"  '{term}': found in {count} chunks")


DETAILED ANALYSIS: Contribution of Expanded Terms

Query: crusade battle Jerusalem

Original terms: battle, crusade, jerusalem
New terms added: lebanon, fight, tussle, palestine, crusader, battles, palestinian, quest, crusades

Contribution of each expanded term:
  'crusader': found in 244 chunks
  'crusades': found in 94 chunks
  'fight': found in 43 chunks
  'palestine': found in 41 chunks
  'battles': found in 13 chunks
  'lebanon': found in 5 chunks
  'palestinian': found in 4 chunks
  'tussle': found in 0 chunks
  'quest': found in 0 chunks

Query: Saladin military victory

Original terms: victory, saladin, military
New terms added: win, armed_forces, triumph, miltary, army, victories

Contribution of each expanded term:
  'army': found in 371 chunks
  'victories': found in 10 chunks
  'win': found in 2 chunks
  'triumph': found in 2 chunks
  'armed_forces': found in 0 chunks
  'miltary': found in 0 chunks

Query: siege fortress attack

Original terms: siege, fortress, attack
New